# DiCE

## Load data

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

df = pd.read_csv("../../datasets/FICO/heloc_dataset_v1.csv")

In [3]:
TARGET = "RiskPerformance"

In [4]:
from sklearn.model_selection import train_test_split

df[TARGET] = df[TARGET].factorize()[0]

train, test = train_test_split(df, test_size=0.25, random_state=42)

## DiCE dataset

In [5]:
import dice_ml
from dice_ml.utils import helpers # helper functions

In [6]:
continuous_features = list(train[df.columns[~train.columns.isin([TARGET])]].columns)

In [7]:
features = {f: [-20, 1000] for f in continuous_features}

In [8]:
d = dice_ml.Data(features=features, outcome_name=TARGET)

## XGBoost

### Load model

In [9]:
from xgboost.sklearn import XGBClassifier

model = XGBClassifier()

In [10]:
model.load_model("../../models/xgboost.json")

In [11]:
import utils

datapoints = pd.read_csv("../../datasets/datapoints.csv")

In [12]:
m = dice_ml.Model(model=model, backend="sklearn")

In [13]:
d.get_decimal_precisions()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

### KD-Tree method

KD-Tree method requires access to the entire dataset.

In [14]:
d = dice_ml.Data(dataframe=train, continuous_features=continuous_features, outcome_name=TARGET)

In [15]:
explainer = dice_ml.Dice(d, m, method="kdtree")

In [16]:
N=100

In [17]:
explanation = explainer.generate_counterfactuals(datapoints, total_CFs=N, desired_class=1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:10<00:00,  7.10s/it]


In [18]:
explanation.visualize_as_dataframe()

Query instance (original outcome : 0)


,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,...,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance,RiskPerformance
0,70,142,9,47,16,0,0,100,-7,7,...,0,2,2,30,55,4,2,1,55,0



Diverse Counterfactual set (new outcome: 1.0)


,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,...,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance,RiskPerformance
8000,70.0,142.0,3.0,62.0,25.0,0.0,0.0,100.0,-7.0,7.0,...,2.0,2.0,2.0,30.0,55.0,4.0,2.0,3.0,55.0,1
5299,75.0,142.0,9.0,47.0,21.0,0.0,0.0,100.0,-7.0,7.0,...,0.0,2.0,2.0,20.0,63.0,4.0,2.0,1.0,55.0,1
6364,76.0,142.0,9.0,47.0,20.0,0.0,0.0,100.0,-7.0,7.0,...,0.0,0.0,0.0,40.0,72.0,4.0,2.0,1.0,55.0,1
6453,71.0,142.0,9.0,49.0,24.0,0.0,0.0,100.0,-7.0,7.0,...,0.0,0.0,0.0,30.0,42.0,4.0,2.0,1.0,55.0,1
324,76.0,142.0,3.0,47.0,32.0,0.0,0.0,100.0,-7.0,7.0,...,0.0,2.0,2.0,16.0,42.0,7.0,2.0,1.0,64.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7542,87.0,142.0,9.0,61.0,16.0,0.0,0.0,100.0,-7.0,7.0,...,-7.0,0.0,0.0,15.0,32.0,1.0,2.0,1.0,55.0,1
8354,78.0,142.0,1.0,47.0,16.0,0.0,0.0,100.0,-7.0,7.0,...,0.0,2.0,2.0,3.0,82.0,4.0,4.0,1.0,55.0,1
6093,83.0,142.0,9.0,64.0,27.0,0.0,0.0,100.0,-7.0,7.0,...,0.0,2.0,2.0,13.0,2.0,4.0,2.0,1.0,40.0,1
1831,83.0,142.0,2.0,47.0,22.0,0.0,0.0,100.0,-7.0,7.0,...,0.0,0.0,0.0,5.0,86.0,4.0,2.0,1.0,55.0,1


Query instance (original outcome : 0)


...

,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,...,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance,RiskPerformance
6941,74.0,161.0,2.0,76.0,16.0,0.0,0.0,88.0,41.0,6.0,...,-8.0,0.0,0.0,28.0,91.0,4.0,1.0,0.0,100.0,1
8648,83.0,161.0,2.0,77.0,12.0,0.0,0.0,92.0,53.0,6.0,...,-8.0,0.0,0.0,12.0,78.0,1.0,1.0,0.0,80.0,1
9209,72.0,161.0,2.0,58.0,22.0,0.0,0.0,91.0,41.0,6.0,...,2.0,1.0,1.0,25.0,80.0,1.0,5.0,0.0,78.0,1
5336,77.0,161.0,2.0,58.0,15.0,0.0,0.0,93.0,30.0,6.0,...,0.0,3.0,3.0,21.0,37.0,1.0,1.0,0.0,44.0,1
9648,82.0,161.0,2.0,85.0,12.0,0.0,0.0,87.0,64.0,6.0,...,-8.0,0.0,0.0,1.0,73.0,1.0,5.0,0.0,86.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3033,75.0,161.0,2.0,58.0,26.0,0.0,0.0,100.0,-7.0,7.0,...,0.0,1.0,1.0,37.0,75.0,4.0,3.0,0.0,58.0,1
10231,77.0,161.0,2.0,58.0,18.0,0.0,0.0,94.0,7.0,4.0,...,10.0,0.0,0.0,13.0,82.0,1.0,4.0,0.0,71.0,1
8100,74.0,121.0,2.0,61.0,21.0,0.0,0.0,95.0,32.0,6.0,...,9.0,0.0,0.0,58.0,91.0,1.0,4.0,0.0,86.0,1
2132,80.0,161.0,8.0,58.0,17.0,0.0,0.0,100.0,-7.0,7.0,...,0.0,1.0,1.0,21.0,91.0,1.0,3.0,0.0,100.0,1


Query instance (original outcome : 0)


,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,...,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance,RiskPerformance
0,65,49,9,23,11,0,0,100,-7,7,...,-7,1,1,76,74,4,2,3,67,0



Diverse Counterfactual set (new outcome: 1.0)


,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,...,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance,RiskPerformance
4987,74.0,49.0,9.0,23.0,11.0,0.0,0.0,100.0,-7.0,7.0,...,23.0,1.0,1.0,41.0,74.0,4.0,2.0,1.0,57.0,1
3104,79.0,49.0,4.0,62.0,28.0,0.0,0.0,100.0,-7.0,7.0,...,1.0,1.0,1.0,26.0,74.0,4.0,2.0,1.0,67.0,1
9396,84.0,49.0,9.0,23.0,11.0,0.0,0.0,100.0,-7.0,7.0,...,0.0,1.0,1.0,1.0,59.0,1.0,2.0,0.0,60.0,1
7759,73.0,80.0,2.0,42.0,11.0,0.0,0.0,100.0,-7.0,7.0,...,-8.0,1.0,1.0,61.0,74.0,4.0,2.0,0.0,88.0,1
4713,76.0,49.0,4.0,23.0,11.0,0.0,0.0,100.0,-7.0,7.0,...,4.0,1.0,1.0,21.0,86.0,4.0,2.0,0.0,54.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10185,82.0,-8.0,4.0,55.0,24.0,0.0,0.0,100.0,-7.0,7.0,...,4.0,1.0,1.0,5.0,58.0,4.0,2.0,1.0,36.0,1
230,73.0,116.0,23.0,65.0,23.0,0.0,0.0,100.0,-7.0,7.0,...,1.0,1.0,1.0,25.0,74.0,4.0,4.0,1.0,67.0,1
9723,76.0,49.0,21.0,23.0,4.0,0.0,0.0,100.0,-7.0,7.0,...,20.0,1.0,1.0,0.0,74.0,0.0,2.0,-8.0,67.0,1
3775,84.0,-8.0,9.0,91.0,17.0,0.0,0.0,100.0,-7.0,7.0,...,3.0,1.0,1.0,11.0,48.0,4.0,2.0,0.0,55.0,1


In [25]:
def create_result_df(_model, _explanation, index: int, method: str, model_type: str):
    exp_df = _explanation.cf_examples_list[index].final_cfs_df
    if "RiskPerformance" in exp_df:
        exp_df = exp_df.drop(columns=["RiskPerformance"])
    probs = _model.predict_proba(exp_df)
    exp_df = exp_df.add_prefix("Cf")
    exp_df['GoalScore'] = probs[:, 1]
    exp_df['GoalValue'] = 1
    exp_df['GoalName'] = "RiskPerformance"
    exp_df['method'] = method
    exp_df['model'] = model_type
    Xd = _explanation.cf_examples_list[index].test_instance_df.to_dict()
    names = list(Xd.keys())
    for i in range(len(names)):
        exp_df.insert(i, names[i], list(Xd[names[i]].values())[0])
    return exp_df

In [29]:
counterfactuals = []
for i in range(10):
    result_kdtree = create_result_df(model, explanation, i, "DiCE-kdtree", "XGBoost")
    counterfactuals.append(result_kdtree)
    
kdtree_final = pd.concat(counterfactuals)

In [30]:
kdtree_final

,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,...,CfNetFractionInstallBurden,CfNumRevolvingTradesWBalance,CfNumInstallTradesWBalance,CfNumBank2NatlTradesWHighUtilization,CfPercentTradesWBalance,GoalScore,GoalValue,GoalName,method,model
8000,70,142,9,47,16,0,0,100,-7,7,...,58.0,4.0,3.0,3.0,58.0,0.767595,1,RiskPerformance,DiCE-kdtree,XGBoost
5299,70,142,9,47,16,0,0,100,-7,7,...,63.0,3.0,2.0,1.0,56.0,0.614679,1,RiskPerformance,DiCE-kdtree,XGBoost
6364,70,142,9,47,16,0,0,100,-7,7,...,72.0,2.0,2.0,1.0,50.0,0.511159,1,RiskPerformance,DiCE-kdtree,XGBoost
6453,70,142,9,47,16,0,0,100,-7,7,...,42.0,6.0,2.0,2.0,50.0,0.500685,1,RiskPerformance,DiCE-kdtree,XGBoost
324,70,142,9,47,16,0,0,100,-7,7,...,42.0,7.0,2.0,1.0,64.0,0.636118,1,RiskPerformance,DiCE-kdtree,XGBoost
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10185,65,49,9,23,11,0,0,100,-7,7,...,58.0,2.0,2.0,1.0,36.0,0.848123,1,RiskPerformance,DiCE-kdtree,XGBoost
230,65,49,9,23,11,0,0,100,-7,7,...,70.0,3.0,4.0,1.0,62.0,0.720880,1,RiskPerformance,DiCE-kdtree,XGBoost
9723,65,49,9,23,11,0,0,100,-7,7,...,67.0,0.0,2.0,-8.0,67.0,0.592765,1,RiskPerformance,DiCE-kdtree,XGBoost
3775,65,49,9,23,11,0,0,100,-7,7,...,48.0,4.0,2.0,0.0,55.0,0.866074,1,RiskPerformance,DiCE-kdtree,XGBoost


### Random sampling

In [31]:
explainer = dice_ml.Dice(d, m, method="random")

In [32]:
explanation = explainer.generate_counterfactuals(datapoints, total_CFs=N, desired_class=1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.03it/s]


In [34]:
counterfactuals = []
for i in range(10):
    result_random = create_result_df(model, explanation, i, "DiCE-random", "XGBoost")
    counterfactuals.append(result_random)
    
random_final = pd.concat(counterfactuals)
random_final

,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,...,CfNetFractionInstallBurden,CfNumRevolvingTradesWBalance,CfNumInstallTradesWBalance,CfNumBank2NatlTradesWHighUtilization,CfPercentTradesWBalance,GoalScore,GoalValue,GoalName,method,model
0,70,142,9,47,16,0,0,100,-7,7,...,55,4,2,-6,55,0.593530,1,RiskPerformance,DiCE-random,XGBoost
1,70,142,9,47,16,0,0,100,-7,7,...,55,4,2,1,55,0.566490,1,RiskPerformance,DiCE-random,XGBoost
2,70,142,9,47,16,0,0,100,-7,7,...,55,4,2,1,55,0.727624,1,RiskPerformance,DiCE-random,XGBoost
3,70,142,9,47,16,0,0,100,-7,7,...,12,4,2,1,55,0.602797,1,RiskPerformance,DiCE-random,XGBoost
4,70,142,9,47,16,0,0,100,-7,7,...,55,-3,2,1,55,0.530053,1,RiskPerformance,DiCE-random,XGBoost
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,65,49,9,23,11,0,0,100,-7,7,...,74,-4,2,3,67,0.505414,1,RiskPerformance,DiCE-random,XGBoost
96,65,49,9,23,11,0,0,100,-7,7,...,74,4,2,3,12,0.537164,1,RiskPerformance,DiCE-random,XGBoost
97,65,49,9,23,11,0,0,100,-7,7,...,74,4,10,3,67,0.546712,1,RiskPerformance,DiCE-random,XGBoost
98,65,49,9,23,11,0,0,100,-7,7,...,74,4,2,3,70,0.622487,1,RiskPerformance,DiCE-random,XGBoost


### Genetic

In [35]:
explainer = dice_ml.Dice(d, m, method="genetic")

In [36]:
explanation = explainer.generate_counterfactuals(datapoints, total_CFs=N, desired_class=1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [06:48<00:00, 40.86s/it]


In [37]:
counterfactuals = []
for i in range(10):
    result_genetic = create_result_df(model, explanation, i, "DiCE-genetic", "XGBoost")
    counterfactuals.append(result_genetic)
    
genetic_final = pd.concat(counterfactuals)
genetic_final

,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,...,CfNetFractionInstallBurden,CfNumRevolvingTradesWBalance,CfNumInstallTradesWBalance,CfNumBank2NatlTradesWHighUtilization,CfPercentTradesWBalance,GoalScore,GoalValue,GoalName,method,model
0,70,142,9,47,16,0,0,100,-7,7,...,17.0,4.0,-9.0,2.0,0.0,0.514380,1,RiskPerformance,DiCE-genetic,XGBoost
0,70,142,9,47,16,0,0,100,-7,7,...,42.0,-9.0,4.0,1.0,-9.0,0.592118,1,RiskPerformance,DiCE-genetic,XGBoost
0,70,142,9,47,16,0,0,100,-7,7,...,-9.0,4.0,3.0,1.0,1.0,0.615547,1,RiskPerformance,DiCE-genetic,XGBoost
0,70,142,9,47,16,0,0,100,-7,7,...,38.0,4.0,2.0,2.0,1.0,0.572715,1,RiskPerformance,DiCE-genetic,XGBoost
0,70,142,9,47,16,0,0,100,-7,7,...,3.0,-9.0,3.0,3.0,0.0,0.557159,1,RiskPerformance,DiCE-genetic,XGBoost
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,65,49,9,23,11,0,0,100,-7,7,...,-9.0,-9.0,3.0,-9.0,3.0,0.503013,1,RiskPerformance,DiCE-genetic,XGBoost
0,65,49,9,23,11,0,0,100,-7,7,...,-9.0,4.0,4.0,3.0,1.0,0.571217,1,RiskPerformance,DiCE-genetic,XGBoost
0,65,49,9,23,11,0,0,100,-7,7,...,27.0,-8.0,5.0,-9.0,-9.0,0.664247,1,RiskPerformance,DiCE-genetic,XGBoost
0,65,49,9,23,11,0,0,100,-7,7,...,4.0,-8.0,2.0,-9.0,0.0,0.505551,1,RiskPerformance,DiCE-genetic,XGBoost


## MLP

### Load model

In [38]:
from joblib import load

mlp_model = load('../../models/mlp.joblib') 

### KD-Tree method

In [39]:
m_mlp = dice_ml.Model(model=mlp_model, backend="sklearn")

In [40]:
explainer = dice_ml.Dice(d, m_mlp, method="kdtree")

In [41]:
explanation = explainer.generate_counterfactuals(datapoints, total_CFs=N, desired_class=1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:41<00:00,  4.13s/it]


In [43]:
counterfactuals = []
for i in range(10):
    result_kdtree = create_result_df(mlp_model, explanation, i, "DiCE-kdtree", "MLP")
    counterfactuals.append(result_kdtree)
    
result_mlp_kdtree = pd.concat(counterfactuals)
result_mlp_kdtree

,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,...,CfNetFractionInstallBurden,CfNumRevolvingTradesWBalance,CfNumInstallTradesWBalance,CfNumBank2NatlTradesWHighUtilization,CfPercentTradesWBalance,GoalScore,GoalValue,GoalName,method,model
8000,70,142,9,47,16,0,0,100,-7,7,...,58.0,4.0,3.0,3.0,58.0,0.679485,1,RiskPerformance,DiCE-kdtree,MLP
5299,70,142,9,47,16,0,0,100,-7,7,...,63.0,3.0,2.0,1.0,56.0,0.785958,1,RiskPerformance,DiCE-kdtree,MLP
6453,70,142,9,47,16,0,0,100,-7,7,...,42.0,6.0,2.0,2.0,50.0,0.652976,1,RiskPerformance,DiCE-kdtree,MLP
324,70,142,9,47,16,0,0,100,-7,7,...,42.0,7.0,2.0,1.0,64.0,0.628889,1,RiskPerformance,DiCE-kdtree,MLP
5161,70,142,9,47,16,0,0,100,-7,7,...,87.0,4.0,2.0,1.0,67.0,0.543491,1,RiskPerformance,DiCE-kdtree,MLP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387,65,49,9,23,11,0,0,100,-7,7,...,-8.0,2.0,2.0,0.0,50.0,0.728100,1,RiskPerformance,DiCE-kdtree,MLP
7726,65,49,9,23,11,0,0,100,-7,7,...,-8.0,4.0,1.0,0.0,56.0,0.650010,1,RiskPerformance,DiCE-kdtree,MLP
3590,65,49,9,23,11,0,0,100,-7,7,...,87.0,4.0,3.0,1.0,50.0,0.775490,1,RiskPerformance,DiCE-kdtree,MLP
10089,65,49,9,23,11,0,0,100,-7,7,...,90.0,3.0,3.0,0.0,67.0,0.634995,1,RiskPerformance,DiCE-kdtree,MLP


### Random sampling

In [44]:
explainer = dice_ml.Dice(d, m_mlp, method="random")

In [45]:
explanation = explainer.generate_counterfactuals(datapoints, total_CFs=N, desired_class=1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.90it/s]


In [46]:
counterfactuals = []
for i in range(10):
    result = create_result_df(mlp_model, explanation, i, "DiCE-random", "MLP")
    counterfactuals.append(result)
    
result_mlp_random = pd.concat(counterfactuals)
result_mlp_random

,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,...,CfNetFractionInstallBurden,CfNumRevolvingTradesWBalance,CfNumInstallTradesWBalance,CfNumBank2NatlTradesWHighUtilization,CfPercentTradesWBalance,GoalScore,GoalValue,GoalName,method,model
0,70,142,9,47,16,0,0,100,-7,7,...,55,4,2,1,55,0.732487,1,RiskPerformance,DiCE-random,MLP
1,70,142,9,47,16,0,0,100,-7,7,...,55,4,2,1,55,0.639378,1,RiskPerformance,DiCE-random,MLP
2,70,142,9,47,16,0,0,100,-7,7,...,59,4,2,1,55,0.724004,1,RiskPerformance,DiCE-random,MLP
3,70,142,9,47,16,0,0,100,-7,7,...,55,4,2,6,66,0.516373,1,RiskPerformance,DiCE-random,MLP
4,70,142,9,47,16,0,0,100,-7,7,...,55,4,2,1,38,0.523074,1,RiskPerformance,DiCE-random,MLP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,65,49,9,23,11,0,0,100,-7,7,...,74,4,2,3,67,0.750509,1,RiskPerformance,DiCE-random,MLP
96,65,49,9,23,11,0,0,100,-7,7,...,74,4,19,3,67,0.883163,1,RiskPerformance,DiCE-random,MLP
97,65,49,9,23,11,0,0,100,-7,7,...,74,4,2,3,67,0.949658,1,RiskPerformance,DiCE-random,MLP
98,65,49,9,23,11,0,0,100,-7,7,...,74,4,2,16,67,0.958566,1,RiskPerformance,DiCE-random,MLP


### Genetic

In [47]:
explainer = dice_ml.Dice(d, m_mlp, method="genetic")

In [48]:
explanation = explainer.generate_counterfactuals(datapoints, total_CFs=N, desired_class=1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [04:23<00:00, 26.34s/it]


In [49]:
counterfactuals = []
for i in range(10):
    result = create_result_df(mlp_model, explanation, i, "DiCE-genetic", "MLP")
    counterfactuals.append(result)
    
result_mlp_genetic = pd.concat(counterfactuals)
result_mlp_genetic

,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,...,CfNetFractionInstallBurden,CfNumRevolvingTradesWBalance,CfNumInstallTradesWBalance,CfNumBank2NatlTradesWHighUtilization,CfPercentTradesWBalance,GoalScore,GoalValue,GoalName,method,model
0,70,142,9,47,16,0,0,100,-7,7,...,25.0,4.0,2.0,3.0,0.0,0.573234,1,RiskPerformance,DiCE-genetic,MLP
0,70,142,9,47,16,0,0,100,-7,7,...,-9.0,4.0,2.0,13.0,1.0,0.756905,1,RiskPerformance,DiCE-genetic,MLP
0,70,142,9,47,16,0,0,100,-7,7,...,22.0,4.0,2.0,3.0,1.0,0.675172,1,RiskPerformance,DiCE-genetic,MLP
0,70,142,9,47,16,0,0,100,-7,7,...,0.0,4.0,5.0,3.0,-9.0,0.664406,1,RiskPerformance,DiCE-genetic,MLP
0,70,142,9,47,16,0,0,100,-7,7,...,48.0,-8.0,1.0,1.0,0.0,0.720580,1,RiskPerformance,DiCE-genetic,MLP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,65,49,9,23,11,0,0,100,-7,7,...,28.0,-9.0,6.0,1.0,-9.0,0.918396,1,RiskPerformance,DiCE-genetic,MLP
0,65,49,9,23,11,0,0,100,-7,7,...,13.0,-8.0,2.0,1.0,0.0,0.611479,1,RiskPerformance,DiCE-genetic,MLP
0,65,49,9,23,11,0,0,100,-7,7,...,44.0,4.0,6.0,3.0,-9.0,0.922261,1,RiskPerformance,DiCE-genetic,MLP
0,65,49,9,23,11,0,0,100,-7,7,...,50.0,-9.0,5.0,3.0,1.0,0.657965,1,RiskPerformance,DiCE-genetic,MLP


## Export results

In [50]:
final_df = pd.concat([kdtree_final, random_final, genetic_final, result_mlp_kdtree, result_mlp_random, result_mlp_genetic])
final_df.T

,8000,5299,6364,6453,324,9986,3407,7245,3521,8151,...,0,0,0,0,0,0,0,0,0,0
ExternalRiskEstimate,70,70,70,70,70,70,70,70,70,70,...,65,65,65,65,65,65,65,65,65,65
MSinceOldestTradeOpen,142,142,142,142,142,142,142,142,142,142,...,49,49,49,49,49,49,49,49,49,49
MSinceMostRecentTradeOpen,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
AverageMInFile,47,47,47,47,47,47,47,47,47,47,...,23,23,23,23,23,23,23,23,23,23
NumSatisfactoryTrades,16,16,16,16,16,16,16,16,16,16,...,11,11,11,11,11,11,11,11,11,11
NumTrades60Ever2DerogPubRec,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NumTrades90Ever2DerogPubRec,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PercentTradesNeverDelq,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
MSinceMostRecentDelq,-7,-7,-7,-7,-7,-7,-7,-7,-7,-7,...,-7,-7,-7,-7,-7,-7,-7,-7,-7,-7
MaxDelq2PublicRecLast12M,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7


In [51]:
final_df.to_csv("../../results/cf-dice.csv")